<a href="https://colab.research.google.com/github/LucasLessa1/X-rayBodyPart/blob/lucas1/Just_The_necessary.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports and Ajusting file


In [1]:
pip install pydicom

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 14.6 MB/s 


In [2]:
import cv2
import pandas as pd
import os
import shutil
import time
import math
import zipfile
import numpy as np
from tqdm import tqdm
import os
from PIL import Image
import string
import matplotlib.pyplot as plt
from scipy.io import loadmat
import pydicom
import gdown
# from google.colab.patches import cv2_imshow

In [3]:
def download(id): 
    url = 'https://drive.google.com/uc?id=' + str(id)
    gdown.download(url, output = None, quiet = False)

def unzip(path): #Função para unzip
    zip = zipfile.ZipFile(path)
    zip.extractall()
    zip.close()

In [4]:
# https://drive.google.com/file/d/1ev-r31j8oRzDlKM_toaeADO2psrA_XXm/view?usp=sharing

download('1ev-r31j8oRzDlKM_toaeADO2psrA_XXm')
#/mnt/c/Códigos_VS/X-rayBodyPart/archive.zip
unzip('/content/archive.zip')

Downloading...
From: https://drive.google.com/uc?id=1ev-r31j8oRzDlKM_toaeADO2psrA_XXm
To: /content/archive.zip
100%|██████████| 269M/269M [00:03<00:00, 84.7MB/s]


In [5]:
train_df = pd.read_csv('/content/train.csv')
test_df = pd.read_csv('/content/sample_submission.csv')

In [6]:
test_df.head(5)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10001001190452685542...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
1,1.2.826.0.1.3680043.8.498.10022667601042710442...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
2,1.2.826.0.1.3680043.8.498.10024395388921105474...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
3,1.2.826.0.1.3680043.8.498.10026689165626095651...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...
4,1.2.826.0.1.3680043.8.498.10035936364561920980...,0 1 2 3 4 5 6 7 8 9 10 11 12 13 14 15 16 17 18...


In [7]:
train_df.head(5)

,SOPInstanceUID,Target
0,1.2.826.0.1.3680043.8.498.10025629581362719970...,0
1,1.2.826.0.1.3680043.8.498.10036150326276641158...,15
2,1.2.826.0.1.3680043.8.498.10038426859954986240...,12
3,1.2.826.0.1.3680043.8.498.10050991192143676483...,14
4,1.2.826.0.1.3680043.8.498.10053309524595490852...,3


# Pre-processing

In [8]:
print("Columns:")
print(train_df.columns, "\n")

print("Types in columns:")
print(train_df.dtypes, "\n")

print("Types in info:")
print(train_df.describe(), "\n")

Columns:
Index(['SOPInstanceUID', 'Target'], dtype='object') 

Types in columns:
SOPInstanceUID    object
Target            object
dtype: object 

Types in info:
                                           SOPInstanceUID Target
count                                                1738   1738
unique                                               1738     41
top     1.2.826.0.1.3680043.8.498.10025629581362719970...     3 
freq                                                    1    724 



## Fixing folders

In [9]:
def dcmtag2table(folder, list_of_tags):

    list_of_tags = list_of_tags.copy()
    items = []
    table = []
    filelist = []

    for root, dirs, files in os.walk(folder, topdown=False):
#Adiciona os caminhos de cada imagem na lista
        for name in files:
            filelist.append(os.path.join(root, name))
    aux= 0
    for _f in filelist:
      #Em cada caminho da imagem(.dcm)
        try:
            ds = pydicom.dcmread(_f, stop_before_pixels=True)
            #Pega o Dataset de cada imagem e ler no próximo for
            items = []
            items.append(_f)
            #Adiciono o caminho em "items"
            for _tag in list_of_tags:
                if _tag in ds:
                  
                    items.append(ds.data_element(_tag).value)
                    #Adiciono a features "PhotometricInterpretation", "BitsAllocated" and "SOPInstanceUID" em "items"
                else:
                    items.append("Not found")
            
            table.append(items)
            #Adiciona tudo em uma outra lista
        except:
            print("Skipping non-DICOM: " + _f)

    df = pd.DataFrame(table, columns=["Filename", "PhotometricInterpretation", "BitsAllocated","SOPInstanceUID"])
    #Crio um data set com todos os caminhos Photometric, Bits and SOP
    return df

*   SOP Instance UID - Unique Identification
*   SOP Class UID    - 

In [10]:
tags = ['PhotometricInterpretation','BitsAllocated', 'SOPInstanceUID' ]
dicom_tags_train =  dcmtag2table('/content/train', tags)
dicom_tags_test = dcmtag2table('/content/test', tags)

In [11]:
dicom_tags_train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID
0,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12914395967183998183...
1,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.96552478101784691863...
2,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.49745423722389558511...
3,/content/train/train/train/449/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.50364199276235309496...
4,/content/train/train/train/812/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.92674478707737111731...
...,...,...,...,...
1733,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.10999331023067486796...
1734,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.74290446148781922284...
1735,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.99203702319483886875...
1736,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.60142885298592397727...


In [12]:
print(dicom_tags_train.PhotometricInterpretation.value_counts())

MONOCHROME1    1659
MONOCHROME2      79
Name: PhotometricInterpretation, dtype: int64


In [13]:
train = dicom_tags_train.merge(train_df, on =  'SOPInstanceUID')
test = dicom_tags_test.merge(test_df,on =  'SOPInstanceUID')

In [14]:
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target
0,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12914395967183998183...,14
1,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.96552478101784691863...,14
2,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.49745423722389558511...,14
3,/content/train/train/train/449/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.50364199276235309496...,3
4,/content/train/train/train/812/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.92674478707737111731...,17


In [15]:
target = list(train["Target"])

In [16]:
#Let's  see our Target distribution

bodyparts = {
0 : 'Abdomen' ,
1 :'Ankle' ,
2 :'Cervical Spine',
3 : 'Chest' ,
4 :'Clavicles' ,
5 :'Elbow' ,
6 :'Feet' ,
7 : 'Finger' ,
8 : 'Forearm' ,
9 : 'Hand' ,
10 : 'Hip' ,
11 : 'Knee' ,
12 : 'Lower Leg' ,
13 : 'Lumbar Spine' ,
14 : 'Others' ,
15 :'Pelvis',
16 :'Shoulder' ,
17 :'Sinus' ,
18 : 'Skull' ,
19 : 'Thigh' ,
20 :'Thoracic Spine',
21: 'Wrist',
}

In [17]:
#Create Label from target
def no_to_label(label):
    label_list_string = []
    trimed_label = label.rstrip()
    label_list = trimed_label.split(" ")
    label_list = [int(i) for i in label_list]
    for label in label_list:
        label_list_string.append(bodyparts[label])
    label_string = ' and '.join(label_list_string)
    return label_string

target_list = train['Target'].tolist()
label_column = []

for label in tqdm(target_list):
    label_string = no_to_label(label)
    label_column.append(label_string)

train['Label'] = label_column
train['Label'].unique()

100%|██████████| 1738/1738 [00:00<00:00, 281314.41it/s]


array(['Others', 'Chest', 'Sinus', 'Hand', 'Wrist', 'Hand and Wrist',
       'Knee', 'Ankle', 'Abdomen', 'Lumbar Spine and Thoracic Spine',
       'Skull', 'Pelvis', 'Cervical Spine', 'Pelvis and Thigh', 'Feet',
       'Ankle and Knee and Lower Leg', 'Thoracic Spine', 'Forearm',
       'Abdomen and Chest', 'Lower Leg', 'Lumbar Spine', 'Clavicles',
       'Shoulder', 'Finger', 'Hip', 'Ankle and Feet', 'Elbow',
       'Cervical Spine and Lumbar Spine and Thoracic Spine',
       'Finger and Hand', 'Thigh', 'Hip and Knee and Thigh',
       'Cervical Spine and Thoracic Spine', 'Elbow and Forearm',
       'Chest and Finger', 'Chest and Feet', 'Abdomen and Pelvis',
       'Hip and Thigh', 'Ankle and Lower Leg', 'Chest and Pelvis',
       'Knee and Lower Leg', 'Forearm and Shoulder'], dtype=object)

In [18]:
train

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
0,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12914395967183998183...,14,Others
1,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.96552478101784691863...,14,Others
2,/content/train/train/train/177/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.49745423722389558511...,14,Others
3,/content/train/train/train/449/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.50364199276235309496...,3,Chest
4,/content/train/train/train/812/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.92674478707737111731...,17,Sinus
...,...,...,...,...,...,...
1733,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.10999331023067486796...,9 21,Hand and Wrist
1734,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.74290446148781922284...,9 21,Hand and Wrist
1735,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.99203702319483886875...,2,Cervical Spine
1736,/content/train/train/train/1331/1.2.826.0.1.36...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.60142885298592397727...,15,Pelvis


In [19]:
train.Label.value_counts()

Chest                                                 724
Others                                                120
Knee                                                  102
Abdomen                                                80
Cervical Spine                                         77
Feet                                                   70
Pelvis                                                 67
Wrist                                                  63
Lumbar Spine and Thoracic Spine                        45
Hand and Wrist                                         42
Ankle                                                  41
Shoulder                                               40
Lumbar Spine                                           39
Hand                                                   31
Elbow                                                  23
Sinus                                                  23
Lower Leg                                              19
Hip           

## Delete every row with more than one organ

In [20]:
frase = "Ankle and Knee and Lower Leg"
if "and" in frase:
    print("Ele identificou")

Ele identificou


In [21]:
aux = list(train["Label"])
index=[]
for i in range(len(aux)):
    if "and" in aux[i]:
        index.append(i)

In [22]:
train.drop(index, axis=0, inplace=True)

In [23]:
train.Label.value_counts()

Chest             724
Others            120
Knee              102
Abdomen            80
Cervical Spine     77
Feet               70
Pelvis             67
Wrist              63
Ankle              41
Shoulder           40
Lumbar Spine       39
Sinus              23
Elbow              23
Lower Leg          19
Hip                19
Thoracic Spine     15
Finger             15
Forearm            12
Skull              10
Clavicles           9
Thigh               7
Name: Label, dtype: int64

In [24]:
train.columns

Index(['Filename', 'PhotometricInterpretation', 'BitsAllocated',
       'SOPInstanceUID', 'Target', 'Label'],
      dtype='object')

In [25]:
# #Creating a Training Folder
# os.mkdir("/mnt/c/Códigos_VS/X-rayBodyPart/" + "TrainingImages" + "/")


In [26]:
train = train.sort_values(by=['Label'])
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
1161,/content/train/train/train/660/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11430185427783861938...,0,Abdomen
481,/content/train/train/train/597/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.90879301558575796376...,0,Abdomen
1167,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11666143137939211262...,0,Abdomen
1168,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.68959812189539083943...,0,Abdomen
1170,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12608809850577365286...,0,Abdomen


#### Creating Folders to each label

In [27]:
def creatingFolder(nameList, path):
    for name in nameList:
        os.mkdir(path + name + '/')



In [28]:
# nameList = train["Label"].unique()
# path = "/mnt/c/Códigos_VS/X-rayBodyPart/TrainingImages/"


# try:
#   creatingFolder(nameList, path)
  
# except FileExistsError:
#   shutil.rmtree(path)


#### Putting each imagem to the specific folder

In [29]:
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
1161,/content/train/train/train/660/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11430185427783861938...,0,Abdomen
481,/content/train/train/train/597/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.90879301558575796376...,0,Abdomen
1167,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11666143137939211262...,0,Abdomen
1168,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.68959812189539083943...,0,Abdomen
1170,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12608809850577365286...,0,Abdomen


In [30]:
train.Label.value_counts()

Chest             724
Others            120
Knee              102
Abdomen            80
Cervical Spine     77
Feet               70
Pelvis             67
Wrist              63
Ankle              41
Shoulder           40
Lumbar Spine       39
Elbow              23
Sinus              23
Lower Leg          19
Hip                19
Finger             15
Thoracic Spine     15
Forearm            12
Skull              10
Clavicles           9
Thigh               7
Name: Label, dtype: int64

In [31]:
folder = "/mnt/c/Códigos_VS/X-rayBodyPart/TrainingImages/"
for root, dirs, files in os.walk(folder, topdown=False):
    print(root)

In [32]:
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
1161,/content/train/train/train/660/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11430185427783861938...,0,Abdomen
481,/content/train/train/train/597/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.90879301558575796376...,0,Abdomen
1167,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11666143137939211262...,0,Abdomen
1168,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.68959812189539083943...,0,Abdomen
1170,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12608809850577365286...,0,Abdomen


In [33]:
# dicom = pydicom.dcmread(filename)
# img = dicom.pixel_array
# im = Image.fromarray(img.astype(np.uint8))
# var = filename.split("/")
# patient = var[5]
# code = var[6]
# im.save("/mnt/c/Códigos_VS/X-rayBodyPart/" + name + "/" + var[-1][:-6] + ".bmp")

### As you can see the data is unbalanced, for this we will oversampling the images

#### Testing with ImageDataGenerator

## ImageDataGenerator

In [34]:
train.head()

,Filename,PhotometricInterpretation,BitsAllocated,SOPInstanceUID,Target,Label
1161,/content/train/train/train/660/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11430185427783861938...,0,Abdomen
481,/content/train/train/train/597/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.90879301558575796376...,0,Abdomen
1167,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.11666143137939211262...,0,Abdomen
1168,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.68959812189539083943...,0,Abdomen
1170,/content/train/train/train/854/1.2.826.0.1.368...,MONOCHROME1,16,1.2.826.0.1.3680043.8.498.12608809850577365286...,0,Abdomen


In [35]:
os.mkdir("/content/" + "TrainingImages" + "/")

In [36]:
folder = "/content/TrainingImages/"
for root, dirs, files in os.walk(folder, topdown=False):
  # print(root)
  aux = root.split("/")[-1]
  print(root)



/content/TrainingImages/


In [37]:
def returPathFolder(namelabel):
    path = ''
    folder = "/content/TrainingImages/"
    for root, dirs, files in os.walk(folder, topdown=False):
        aux = root.split("/")[-1]
        if aux == namelabel:
            path = root
            break
    # print(path)

    return path
x=returPathFolder("Finger")
x

''

### Part of oversampling

In [38]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
import numpy as np
import cv2
import os

In [39]:
train_data = []
train_label = []

pathList = list(train['Filename'])
labeList = list(train['Target'])

train_label = labeList
for path in pathList:
    dicom = pydicom.dcmread(path)
    img = dicom.pixel_array
    # im = Image.fromarray(img.astype(np.uint8))
    train_data.append(img.astype(np.uint8))


In [40]:
test_data = []
test_label = []

pathList = list(test['Filename'])
labeList = list(test['Target'])

test_label = labeList
for path in pathList:
    dicom = pydicom.dcmread(path)
    img = dicom.pixel_array
    # im = Image.fromarray(img.astype(np.float))
    test_data.append(img.astype(np.uint8))


In [41]:
train_data = np.array(train_data, dtype="float") / 255.0
test_data = np.array(test_data, dtype="float") / 255.0

#This label enconder put the label in a code 
le = LabelEncoder()
train_label = le.fit_transform(train_label)
train_label = to_categorical(train_label)
(trainX, valX, trainY, valY) = train_test_split(train_data, train_label, test_size=0.20, random_state=42)

In [42]:
train_datagen = ImageDataGenerator(
        rotation_range=20,
        zoom_range=0.15,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.15,
        horizontal_flip=True,
        fill_mode="nearest")

test_datagen = ImageDataGenerator()

In [43]:
train_data.shape

(1575, 256, 256)

In [44]:
batch_size = 8
train_generator = train_datagen.flow(trainX, trainY, batch_size=batch_size)
valid_generator = train_datagen.flow(trainX, trainY, batch_size=batch_size)
test_generator = test_datagen.flow(test_data, batch_size=1)

ValueError: ignored